# Yöntem 2: RNN için Öznitelik Mühendisliği ve Veri Hazırlığı

Bu notebook'un amacı, RNN (Tekrarlayan Sinir Ağları) tabanlı zaman serisi modelleri için veri hazırlığı yapmaktır. Süreç, ham enerji ve hava durumu verilerinin yüklenmesi, hedef değişkenlerin oluşturulması, kapsamlı bir öznitelik mühendisliği ve son olarak verinin RNN modellerinin beklediği dizi (sequence) formatına dönüştürülmesini içerir.

### 1. Kütüphanelerin Yüklenmesi ve Veri Setlerinin Okunması

İlk adımda, veri işleme ve analiz için gerekli olan `pandas`, `numpy` gibi temel kütüphaneler ile ham veri setlerimiz (`energy_dataset.csv` ve `weather_features.csv`) projeye dahil edilir.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


energy = pd.read_csv('../../data/raw/energy_dataset.csv')
weather = pd.read_csv('../../data/raw/weather_features.csv')

### 2. Hedef Değişkenlerin (Target) Tanımlanması

Modellerimizi eğitmek için iki farklı hedef değişkeni tanımlıyoruz. Bu değişkenleri oluştururken gelecekteki bilgiyi sızdırmamak (data leakage) kritik öneme sahiptir. Bu nedenle tüm işlemler, veriyi belirli bir zaman adımı kadar ileri kaydıran `shift()` fonksiyonu ile yapılır.

- **`target_price_1h`**: Bir sonraki saatin elektrik fiyatını tahmin etmek için kullanılır.
- **`target_price_next_day`**: Bir sonraki günün ortalama fiyatını tahmin etmek için kullanılır. Bu, 24 saat sonrası için 24 saatlik bir yuvarlanan ortalama alınarak hesaplanır.

Hesaplama sonrası oluşan `NaN` (boş) değerlere sahip satırlar veri setinden çıkarılır.

In [2]:
energy = energy.sort_values('time').reset_index(drop=True)
energy['time'] = pd.to_datetime(energy['time'], utc=True)

# 1 saat sonrası spot fiyat (her satır için geçerli)
energy['target_price_1h'] = energy['price actual'].shift(-1)

# 1 gün sonrası ortalama fiyat hedefi (sızıntısız)
energy['target_price_next_day'] = (
    energy['price actual']
    .shift(-24)
    .rolling(window=24)
    .mean()
)

#hedefin tanımlanmadığı veriler çıkarılır
energy = energy.dropna(subset=['target_price_1h'])
energy = energy.dropna(subset=['target_price_next_day'])


### 3. Öznitelik Mühendisliği (Feature Engineering)

Modelin performansı büyük ölçüde girdi olarak kullanılan özniteliklerin kalitesine bağlıdır. Bu bölümde, ham verilerden modelin desenleri daha kolay öğrenebilmesi için çeşitli bilgilendirici öznitelikler türeteceğiz.

#### 3.1. Zaman Tabanlı Öznitelikler

Zaman serisindeki döngüsel (cyclical) desenleri (gün içi, hafta içi, yıl içi) yakalamak için tarih ve saat bilgisinden yeni öznitelikler oluşturulur. Sinüs/Kosinüs dönüşümleri, modelin zamanın döngüsel doğasını (örneğin, 23:00'dan sonra 00:00'ın gelmesi) sayısal olarak anlamasına yardımcı olur.

In [3]:
energy['weekday'] = energy['time'].dt.weekday
energy['month'] = energy['time'].dt.month
energy['day'] = energy['time'].dt.day
energy['hour'] = energy['time'].dt.hour
energy['weekofyear'] = energy['time'].dt.isocalendar().week
energy['year'] = energy['time'].dt.year

energy['hour_sin'] = np.sin(2 * np.pi * energy['hour'] / 24)
energy['hour_cos'] = np.cos(2 * np.pi * energy['hour'] / 24)

energy['month_sin'] = np.sin(2 * np.pi * energy['month'] / 12)
energy['month_cos'] = np.cos(2 * np.pi * energy['month'] / 12)

#sadece gece 00:00 için
energy['is_midnight'] = (energy['hour'] == 0).astype(int)

energy['is_valid_for_daily_model'] = energy['target_price_next_day'].notna()

#hafta sonu bilgisi
energy['is_weekend'] = energy['weekday'].isin([5, 6]).astype(int)
energy['is_weekday'] = (energy['weekday'] < 5).astype(int)

# 1: kış, 2: ilkbahar, 3: yaz, 4: sonbahar
def get_season(month):
    return (month % 12 + 3) // 3
energy['season'] = energy['month'].apply(get_season)



#### 3.2. Gecikme Öznitelikleri (Lag Features)

Zaman serisi verilerinde bir değer, genellikle önceki değerlerle yakından ilişkilidir (otokorelasyon). Fiyat (`price actual`) ve tüketim (`total load actual`) için geçmiş değerleri öznitelik olarak eklemek, modelin bu zamansal bağımlılığı öğrenmesini sağlar. Saatlik ve günlük modellerin ihtiyaçlarına göre farklı gecikme pencereleri (1 saat, 24 saat, 1 hafta vb.) kullanılır.

In [4]:
lags_common=[24,168]

#saatlik çözünürlük için
lags_hourly = [1, 2, 3]
for lag in lags_hourly+ lags_common:
    energy[f'price_lag_{lag}'] = energy['price actual'].shift(lag)
    energy[f'load_lag_{lag}'] = energy['total load actual'].shift(lag)

#daily tahmin için daha uzun vadeli set
lags_daily = [48, 72, 96, 120, 144, 168]
for lag in lags_daily+ lags_common:
    energy[f'price_lag_{lag}'] = energy['price actual'].shift(lag)
    energy[f'load_lag_{lag}'] = energy['total load actual'].shift(lag)


#### 3.3. Yuvarlanan Pencere İstatistikleri (Rolling Window Features)

Geçmiş veriler üzerinden belirli bir pencere boyutunda hesaplanan istatistikler (ortalama, standart sapma vb.), serideki yerel trendleri ve oynaklığı (volatilite) yakalamak için kullanılır. Örneğin, 24 saatlik yuvarlanan ortalama, son bir günün ortalama fiyat eğilimini gösterir.

In [5]:
#Rolling Özellikler
rolling_windows_hourly = [3, 6, 12]
rolling_windows_common = [24]
rolling_windows_daily = [48, 72, 168]

#Saatlik rolling – kısa vadeli dalgalanmalar
for window in rolling_windows_hourly:
    energy[f'price_roll_mean_{window}'] = energy['price actual'].shift(1).rolling(window).mean()
    energy[f'price_roll_std_{window}'] = energy['price actual'].shift(1).rolling(window).std()
    energy[f'load_roll_mean_{window}'] = energy['total load actual'].shift(1).rolling(window).mean()
    energy[f'load_roll_std_{window}'] = energy['total load actual'].shift(1).rolling(window).std()

#Ortak rolling
for window in rolling_windows_common:
    energy[f'price_roll_mean_{window}'] = energy['price actual'].shift(1).rolling(window).mean()
    energy[f'price_roll_std_{window}'] = energy['price actual'].shift(1).rolling(window).std()
    energy[f'load_roll_mean_{window}'] = energy['total load actual'].shift(1).rolling(window).mean()
    energy[f'load_roll_std_{window}'] = energy['total load actual'].shift(1).rolling(window).std()

#Günlük model için uzun vadeli
for window in rolling_windows_daily:
    energy[f'price_roll_mean_{window}'] = energy['price actual'].shift(1).rolling(window).mean()
    energy[f'price_roll_std_{window}'] = energy['price actual'].shift(1).rolling(window).std()
    energy[f'load_roll_mean_{window}'] = energy['total load actual'].shift(1).rolling(window).mean()
    energy[f'load_roll_std_{window}'] = energy['total load actual'].shift(1).rolling(window).std()


#### 3.4. Trend ve Momentum Öznitelikleri

Bu öznitelikler, fiyat serisindeki anlık değişimleri ve yönelimleri yakalamayı hedefler.

- **`price_trend_vs_mean`**: Anlık fiyatın, kendi yuvarlanan ortalamasından ne kadar saptığını gösterir.
- **`diff` / `delta` / `pct_change`**: Fiyatın bir önceki zaman adımına göre ne kadar (mutlak veya yüzdesel) değiştiğini gösterir. Bu, momentumu ölçmek için kullanılır.

In [6]:
trend_windows = rolling_windows_hourly + rolling_windows_common + rolling_windows_daily
for window in trend_windows:
    mean_col = f'price_roll_mean_{window}'
    if mean_col in energy.columns:
        energy[f'price_trend_vs_mean_{window}'] = energy['price actual'] - energy[mean_col]

In [7]:
#momentum özellikleri
energy['price_diff_1']= energy['price actual'] - energy['price_lag_1']
energy['load_diff_1']= energy['total load actual'] - energy['load_lag_1']

# Fiyat delta sinyalleri
energy['price_delta_1h_hourly'] = energy['price actual'].diff(1)
energy['price_delta_24h'] = energy['price actual'].diff(24)

# Yüzdesel değişim (ekonometrik dönüşüm)
energy['price_pct_change_1h_hourly'] = energy['price actual'].pct_change(1)
energy['price_pct_change_24h'] = energy['price actual'].pct_change(24)

energy['price_delta_24h_daily'] = energy['price actual'].diff(24)
energy['price_delta_168h_daily'] = energy['price actual'].diff(168)
energy['price_pct_change_168h_daily'] = energy['price actual'].pct_change(168)

#### 3.5. Bir Önceki Güne Ait İstatistikler

Modelin, bir önceki günün genel fiyat davranışını (ortalama, min, maks, standart sapma) bilmesi, tahmin yaparken faydalı bir bağlam sağlar. Bu adımda, her gün için bir önceki günün özet istatistikleri hesaplanır ve mevcut veri setine eklenir.

In [8]:
energy_indexed = energy.set_index('time')

# Günlük istatistikleri hesapla ve bir gün ileri kaydır
daily_stats = energy_indexed['price actual'].resample('D').agg(['mean', 'std', 'min', 'max']).shift(1)

# Sütun isimlerini daha anlaşılır yap
daily_stats = daily_stats.add_prefix('daily_price_prev_day_')

daily_stats = daily_stats.add_prefix('daily_price_prev_day_')

# Saatlik veriye gün bazında merge et (her saate bir önceki günün istatistikleri eklenecek)
# Önce energy dataframe'ine bir tarih sütunu ekleyelim
energy['date'] = energy['time'].dt.date

daily_stats.index = daily_stats.index.date

# Merge işlemi
energy = pd.merge(energy, daily_stats, left_on='date', right_index=True, how='left')

energy.drop(columns=['date'], inplace=True)

ffill_cols = [col for col in energy.columns if 'daily_price_prev_day' in col]
energy[ffill_cols] = energy[ffill_cols].fillna(method='ffill')

#### 3.6. Dağılım ve Aykırı Değer Tespiti

- **Log Dönüşümü**: Fiyat gibi sağa çarpık dağılımları normale yaklaştırmak için `log1p` dönüşümü uygulanır. Bu, bazı modellerin performansını artırabilir.
- **Aykırı Değer Tespiti**: Fiyattaki ani ve beklenmedik sıçramaları (outlier) belirlemek için IQR (Interquartile Range) yöntemi kullanılır. Bu bilgi, modele aykırı durumları bir öznitelik olarak sunar.

In [9]:
# Log dönüşüm
energy['log_price'] = np.log1p(energy['price actual'])

# günlük için 7 gün 
q1_168 = energy['price actual'].shift(1).rolling(window=168).quantile(0.25)
q3_168 = energy['price actual'].shift(1).rolling(window=168).quantile(0.75)
iqr_168 = q3_168 - q1_168
lower_168 = q1_168 - 1.5 * iqr_168
upper_168 = q3_168 + 1.5 * iqr_168
energy['is_price_outlier_168'] = ((energy['price actual'] < lower_168) | (energy['price actual'] > upper_168)).astype(int)

#saatlik için 24 saatlik pencere
q1_24 = energy['price actual'].shift(1).rolling(window=24).quantile(0.25)
q3_24 = energy['price actual'].shift(1).rolling(window=24).quantile(0.75)
iqr_24 = q3_24 - q1_24
lower_24 = q1_24 - 1.5 * iqr_24
upper_24 = q3_24 + 1.5 * iqr_24
energy['is_price_outlier_24'] = ((energy['price actual'] < lower_24) | (energy['price actual'] > upper_24)).astype(int)

energy['is_price_outlier'] = energy['is_price_outlier_24'] | energy['is_price_outlier_168']

#### 3.7. Sinyal İşleme Tabanlı Öznitelikler (FFT ve Dalgacık)

Fiyat serisini bir sinyal olarak kabul ederek, frekans domeninden öznitelikler çıkarılır.

- **Hızlı Fourier Dönüşümü (FFT)**: Zaman serisindeki baskın periyodiklikleri (örn. 24 saatlik, 1 haftalık döngüler) ortaya çıkarır. `fft_mean`, `fft_std`, `fft_max` gibi istatistikler, bu frekansların gücü hakkında bilgi verir.
- **Haar Dalgacık Dönüşümü (DWT)**: Hem zaman hem de frekans bilgisi sağlar. Bu sayede sinyaldeki ani değişimlerin veya yerel desenlerin ne zaman meydana geldiğini yakalayabilir.

In [10]:
def compute_rolling_fft(series, window, shift=1):
    result = {'fft_mean': [], 'fft_std': [], 'fft_max': []}
    for i in range(len(series)):
        if i < window + shift:
            result['fft_mean'].append(np.nan)
            result['fft_std'].append(np.nan)
            result['fft_max'].append(np.nan)
        else:
            window_data = series.iloc[i - window - shift:i - shift]  # GELECEK VERİ KULLANILMAZ
            fft_vals = np.fft.fft(window_data.fillna(0).values)
            fft_mag = np.abs(fft_vals)
            result['fft_mean'].append(fft_mag.mean())
            result['fft_std'].append(fft_mag.std())
            result['fft_max'].append(fft_mag.max())
    return result

# Saatlik FFT kısa süreli frekanslar için
fft_hourly = compute_rolling_fft(energy['price actual'], window=24)
energy['fft_mean_24'] = fft_hourly['fft_mean']
energy['fft_std_24'] = fft_hourly['fft_std']
energy['fft_max_24'] = fft_hourly['fft_max']

# Günlük FFT daha yumuşak paternleri anlamak için
fft_daily = compute_rolling_fft(energy['price actual'], window=72)
energy['fft_mean_72'] = fft_daily['fft_mean']
energy['fft_std_72'] = fft_daily['fft_std']
energy['fft_max_72'] = fft_daily['fft_max']


#### 3.8. Enerji Üretim Öznitelikleri

Elektrik fiyatı, arz (üretim) ve talep (tüketim) dengesine göre belirlenir. Bu nedenle, üretim kaynaklarına dayalı öznitelikler eklenir.

- **Üretim Türleri**: Fosil ve yenilenebilir kaynaklar gruplandırılır.
- **Toplam ve Oransal Üretim**: Toplam üretim içindeki fosil ve yenilenebilir enerji payları hesaplanır.
- **Üretim Çeşitliliği (Entropi)**: Üretim kaynaklarının ne kadar çeşitli olduğunu ölçer. Yüksek entropi, dengeli bir kaynak dağılımını ifade eder.

In [11]:
fossil_cols = [
    'generation fossil gas',
    'generation fossil hard coal',
    'generation fossil brown coal/lignite',
    'generation fossil oil',
    'generation fossil oil shale',
    'generation fossil coal-derived gas',
    'generation fossil peat'
]

renewable_cols = [
    'generation hydro water reservoir',
    'generation hydro run-of-river and poundage',
    'generation hydro pumped storage consumption',
    'generation wind onshore',
    'generation wind offshore',
    'generation solar',
    'generation biomass',
    'generation geothermal',
    'generation marine',
    'generation other renewable'
]

other_cols = ['generation nuclear', 'generation waste', 'generation other']


In [12]:
#üretim verilerini 1 saat gecikmeli alıyoruz sızıntı riskine karşı
for col in fossil_cols + renewable_cols + other_cols:
    energy[f'{col}_lag1'] = energy[col].shift(1)

lagged_cols = [f'{col}_lag1' for col in fossil_cols + renewable_cols + other_cols]

# Toplam üretim
energy['generation_total'] = energy[[f'{col}_lag1' for col in fossil_cols + renewable_cols + other_cols]].sum(axis=1)

energy['fossil_total'] = energy[[f'{col}_lag1' for col in fossil_cols]].sum(axis=1)
energy['renewable_total'] = energy[[f'{col}_lag1' for col in renewable_cols]].sum(axis=1)

# Oranlar
energy['fossil_ratio'] = energy['fossil_total'] / (energy['generation_total'] + 1e-6)
energy['renewable_ratio'] = energy['renewable_total'] / (energy['generation_total'] + 1e-6)

energy['renewable_diff_1h'] = energy['renewable_total'].diff(1)
energy['renewable_ratio_change'] = energy['renewable_ratio'].diff(1)


# Entropi
def entropy(row):
    p = row / (row.sum() + 1e-6)
    return -(p * np.log(p + 1e-6)).sum()



energy['generation_entropy'] = energy[lagged_cols].apply(entropy, axis=1)

from math import log
N = len(lagged_cols)
energy['generation_entropy_norm'] = energy['generation_entropy'] / log(N)



In [13]:
constant_cols = [col for col in energy.columns if energy[col].nunique() <= 1]

manual_exclude_cols = [
    'generation fossil coal-derived gas',
    'generation fossil oil shale',
    'generation fossil peat',
    'generation geothermal',
    'generation hydro pumped storage aggregated',
    'generation marine',
    'generation wind offshore',
    'forecast wind offshore eday ahead'
]
columns_to_exclude = list(set(constant_cols + manual_exclude_cols))
numeric_cols = energy.drop(columns=constant_cols, errors='ignore').select_dtypes(include=[np.number])
# Sadece sayısal sütunlar içinden korelasyon analizi
if 'target_price_1h' in numeric_cols.columns:
    target_corr_1h = numeric_cols.corr()['target_price_1h'].sort_values(ascending=False)
    print("target_price_1h ile en yüksek korelasyonlu ilk 20 değişken:")
    display(target_corr_1h.head(20))

if 'target_price_next_day' in numeric_cols.columns:
    target_corr_day = numeric_cols.corr()['target_price_next_day'].sort_values(ascending=False)
    print("target_price_next_day ile en yüksek korelasyonlu ilk 20 değişken:")
    display(target_corr_day.head(20))

target_price_1h ile en yüksek korelasyonlu ilk 20 değişken:


target_price_1h                                   1.000000
price actual                                      0.966796
log_price                                         0.940286
price_lag_1                                       0.900505
price_roll_mean_3                                 0.837835
price_lag_2                                       0.821607
target_price_next_day                             0.819545
price_roll_mean_24                                0.803069
price_lag_24                                      0.802385
fft_max_24                                        0.795417
fft_std_24                                        0.795011
price_roll_mean_12                                0.778701
price_roll_mean_6                                 0.776983
price_lag_168                                     0.766842
price_roll_mean_48                                0.763320
price_lag_3                                       0.744609
price_roll_mean_72                                0.7443

target_price_next_day ile en yüksek korelasyonlu ilk 20 değişken:


target_price_next_day                             1.000000
price_roll_mean_24                                0.867450
price_roll_mean_12                                0.861180
fft_max_24                                        0.861140
fft_std_24                                        0.860927
price_roll_mean_168                               0.850980
price_roll_mean_48                                0.842271
price_roll_mean_72                                0.836635
fft_std_72                                        0.834841
fft_max_72                                        0.833541
price_roll_mean_6                                 0.830913
target_price_1h                                   0.819545
price actual                                      0.811131
price_roll_mean_3                                 0.810606
daily_price_prev_day_daily_price_prev_day_mean    0.806693
price_lag_1                                       0.803046
log_price                                         0.7970

In [14]:
import pandas as pd

numeric_cols = energy.select_dtypes(include=[np.number]).columns.tolist()

#hedef sütunları hariç tutuyoruz
exclude_targets = ['target_price_1h', 'target_price_next_day', 
                   'log_target_price_1h', 'log_target_price_next_day']

features = [col for col in numeric_cols if col not in exclude_targets]

#korelasyon hesaplıyoruz
corr_1h = energy[features + ['target_price_1h']].corr()['target_price_1h'].drop('target_price_1h')
corr_day = energy[features + ['target_price_next_day']].corr()['target_price_next_day'].drop('target_price_next_day')

# DataFrame
feature_meta = pd.DataFrame({
    'feature_name': features,
    'corr_1h': corr_1h,
    'corr_next_day': corr_day
})

#kullanım etiketi
def tag_feature(row):
    if abs(row['corr_1h']) >= 0.75 and abs(row['corr_next_day']) >= 0.75:
        return 'both'
    elif abs(row['corr_1h']) >= 0.75:
        return 'hourly_only'
    elif abs(row['corr_next_day']) >= 0.75:
        return 'daily_only'
    else:
        return 'weak'

feature_meta['used_in_model'] = feature_meta.apply(tag_feature, axis=1)

feature_meta.to_csv('../../data/processed/feature_metadata_2.csv', index=False)


#### 3.9. Dışsal (Exogenous) Hava Durumu Öznitelikleri

Hava durumu, hem enerji tüketimini (ısıtma/soğutma) hem de yenilenebilir enerji üretimini (güneş/rüzgar) doğrudan etkiler. Bu nedenle Madrid şehri verileri, genel bir gösterge olarak kullanılarak zaman ekseninde birleştirilir ve bu verilerden yeni öznitelikler (sıcaklık farkları, yuvarlanan nem istatistikleri vb.) türetilir.

In [15]:
weather['dt_iso'] = pd.to_datetime(weather['dt_iso'], utc=True)

In [16]:
madrid_weather = weather[weather['city_name'] == 'Madrid'].copy()
madrid_weather['dt_iso'] = pd.to_datetime(madrid_weather['dt_iso'], utc=True)

#prefix
madrid_weather = madrid_weather.add_prefix('madrid_')
madrid_weather = madrid_weather.rename(columns={'madrid_dt_iso': 'time'})

energy = pd.merge(energy, madrid_weather, on='time', how='left')



In [17]:
print("Weather birleştirildi. Yeni shape:", energy.shape)

Weather birleştirildi. Yeni shape: (36220, 169)


In [18]:
#geçmişe göre flagli yağmur kar durumu
energy['madrid_is_rainy'] = (energy['madrid_rain_3h'].shift(1) > 0).astype(int)
energy['madrid_is_snowy'] = (energy['madrid_snow_3h'].shift(1) > 0).astype(int)

# 3 saat öncesine göre fark sıcaklık
energy['madrid_temp_diff_3h'] = energy['madrid_temp'] - energy['madrid_temp'].shift(3)

# trend yakalama
energy['madrid_temp_roll3_mean'] = energy['madrid_temp'].shift(1).rolling(window=3, min_periods=1).mean()

# 1 haftalık nem dalgalanması
energy['madrid_humidity_roll24_std'] = energy['madrid_humidity'].shift(1).rolling(window=24, min_periods=1).std()

#wind ve pressure
energy['madrid_wind_speed_roll6_mean'] = energy['madrid_wind_speed'].shift(1).rolling(6, min_periods=1).mean()
energy['madrid_pressure_roll12_std'] = energy['madrid_pressure'].shift(1).rolling(12, min_periods=1).std()


In [19]:
weather_cols = [
    col for col in energy.columns 
    if col.startswith('madrid_') and np.issubdtype(energy[col].dtype, np.number)
]

if 'target_price_1h' in energy.columns:
    print("\nHava durumu – target_price_1h korelasyonu:")
    display(energy[weather_cols].corrwith(energy['target_price_1h']).sort_values(ascending=False).abs().head(10))

if 'target_price_next_day' in energy.columns:
    print("\nHava durumu – target_price_next_day korelasyonu:")
    display(energy[weather_cols].corrwith(energy['target_price_next_day']).sort_values(ascending=False).abs().head(10))


Hava durumu – target_price_1h korelasyonu:


madrid_temp_diff_3h           0.108923
madrid_temp_max               0.078584
madrid_temp                   0.069565
madrid_temp_min               0.047953
madrid_temp_roll3_mean        0.044648
madrid_pressure               0.019889
madrid_pressure_roll12_std    0.011486
madrid_humidity_roll24_std    0.007549
madrid_weather_id             0.000874
madrid_snow_3h                0.007969
dtype: float64


Hava durumu – target_price_next_day korelasyonu:


madrid_pressure_roll12_std    0.031569
madrid_weather_id             0.025445
madrid_pressure               0.023418
madrid_temp                   0.015032
madrid_temp_roll3_mean        0.014897
madrid_temp_max               0.012240
madrid_temp_min               0.006878
madrid_humidity_roll24_std    0.004853
madrid_temp_diff_3h           0.001237
madrid_humidity               0.000468
dtype: float64

In [20]:
from scipy.fftpack import fft
import numpy as np

def compute_rolling_fft_features(series, window=24, n_freq=5):
    features = {'fft_energy': [], **{f'fft_peak_{i+1}': [] for i in range(n_freq)}}
    shifted_series = series.shift(1)

    for i in range(len(series)):
        if i < window:
            for k in features:
                features[k].append(np.nan)
        else:
            window_data = shifted_series.iloc[i - window:i].fillna(method="ffill").fillna(method="bfill")
            fft_vals = np.abs(fft(window_data.values))
            fft_energy = np.sum(fft_vals**2)
            top_freqs = np.sort(fft_vals)[-n_freq:]
            for j in range(n_freq):
                features[f'fft_peak_{j+1}'].append(top_freqs[j])
            features['fft_energy'].append(fft_energy)

    return pd.DataFrame(features, index=series.index)

fft_df = compute_rolling_fft_features(energy['price actual'], window=24, n_freq=5)
energy = energy.join(fft_df)


In [21]:
def compute_rolling_haar_features(series, window=24):
    def haar_transform(series):
        n = len(series) // 2 * 2
        approx = [(series[i] + series[i+1]) / 2 for i in range(0, n, 2)]
        detail = [(series[i] - series[i+1]) / 2 for i in range(0, n, 2)]
        return np.mean(approx), np.std(approx), np.mean(detail), np.std(detail)

    shifted_series = series.shift(1)
    features = {'dwt_approx_mean': [], 'dwt_approx_std': [], 'dwt_detail_mean': [], 'dwt_detail_std': []}

    for i in range(len(series)):
        if i < window:
            for k in features:
                features[k].append(np.nan)
        else:
            window_data = shifted_series.iloc[i - window:i].fillna(method="ffill").fillna(method="bfill").values
            a_mean, a_std, d_mean, d_std = haar_transform(window_data)
            features['dwt_approx_mean'].append(a_mean)
            features['dwt_approx_std'].append(a_std)
            features['dwt_detail_mean'].append(d_mean)
            features['dwt_detail_std'].append(d_std)

    return pd.DataFrame(features, index=series.index)

dwt_df = compute_rolling_haar_features(energy['price actual'], window=24)
energy = energy.join(dwt_df)

In [22]:
#korelasyon tablosu ve metadata
numeric_cols = energy.select_dtypes(include=[np.number]).columns.tolist()
targets = ['target_price_1h', 'target_price_next_day']
features = [col for col in numeric_cols if col not in targets + ['log_target_price_1h', 'log_target_price_next_day']]

#korelasyonlar
corr_1h = energy[features + ['target_price_1h']].corr()['target_price_1h'].drop('target_price_1h')
corr_day = energy[features + ['target_price_next_day']].corr()['target_price_next_day'].drop('target_price_next_day')

feature_meta = pd.DataFrame({
    'feature_name': features,
    'corr_1h': corr_1h,
    'corr_next_day': corr_day
})

def tag_feature(row):
    if abs(row['corr_1h']) >= 0.75 and abs(row['corr_next_day']) >= 0.75:
        return 'both'
    elif abs(row['corr_1h']) >= 0.75:
        return 'hourly_only'
    elif abs(row['corr_next_day']) >= 0.75:
        return 'daily_only'
    else:
        return 'weak'

feature_meta['used_in_model'] = feature_meta.apply(tag_feature, axis=1)

#feature tip etiketleme
def get_feature_type(f):
    if 'lag' in f:
        return 'lag'
    elif 'roll_mean' in f:
        return 'rolling_mean'
    elif 'roll_std' in f:
        return 'rolling_std'
    elif 'fft_peak' in f or 'fft_energy' in f:
        return 'fft'
    elif 'dwt_' in f:
        return 'wavelet'
    elif 'entropy' in f:
        return 'generation_entropy'
    elif 'ratio' in f:
        return 'generation_ratio'
    elif 'madrid' in f:
        return 'weather'
    elif 'diff' in f or 'delta' in f:
        return 'momentum'
    elif 'outlier' in f:
        return 'outlier'
    else:
        return 'other'

feature_meta['feature_type'] = feature_meta['feature_name'].apply(get_feature_type)

# Kaydet
feature_meta.to_csv('../../data/processed/feature_metadata_2.csv', index=False)


In [23]:
feature_meta['used_in_model_1h'] = feature_meta['used_in_model'].isin(['both', 'hourly_only'])
feature_meta['used_in_model_next_day'] = feature_meta['used_in_model'].isin(['both', 'daily_only'])

feature_meta.to_csv('../../data/processed/feature_metadata_2.csv', index=False)


In [24]:
TARGET_COLS = ['target_price_1h', 'target_price_next_day']
ID_COLS = ['time']

LEAKY_SOURCE_COLS = ['price actual', 'log_price']


In [25]:
potential_features = [
    col for col in energy.select_dtypes(include=np.number).columns
    if col not in TARGET_COLS + ID_COLS + LEAKY_SOURCE_COLS
]
print(f"Potansiyel feature sayısı: {len(potential_features)}")

Potansiyel feature sayısı: 176


In [26]:
def define_feature_type(col_name):
    # Sizin oluşturduğunuz özel lag/rolling pencerelerine göre
    if any(f in col_name for f in ['lag_1', 'lag_2', 'roll_mean_3', 'roll_std_3', 'roll_mean_6', 'roll_std_6', 'roll_mean_12', 'roll_std_12']):
        return 'hourly_specific'
    if any(f in col_name for f in ['lag_48', 'lag_72', 'lag_96', 'lag_120', 'lag_144', 'roll_mean_48', 'roll_std_48', 'roll_mean_72', 'roll_std_72']):
        return 'daily_specific'
    
    # Genel kategoriler
    if 'fft' in col_name: return 'fft'
    if 'dwt' in col_name: return 'wavelet'
    if 'madrid' in col_name: return 'weather'
    if 'entropy' in col_name: return 'generation_entropy'
    if 'ratio' in col_name: return 'generation_ratio'
    if any(k in col_name for k in ['diff', 'delta', 'pct_change']): return 'momentum'
    
    # kalanlar ortak kabul edilir
    return 'common'

In [27]:
feature_meta = pd.DataFrame({'feature_name': potential_features})
feature_meta['feature_type'] = feature_meta['feature_name'].apply(define_feature_type)

### 4. Öznitelik Seçimi ve Temizleme

Türetilen çok sayıda öznitelik arasından gürültülü, gereksiz veya sızıntıya neden olabilecek olanları elemek için sistematik bir temizleme yapılır.

- **Yüksek Korelasyonlu Özniteliklerin Elenmesi**: Birbiriyle %98'den daha yüksek korelasyona sahip öznitelik çiftlerinden, hedef değişkenle daha az ilişkili olanı çıkarılır. Bu, *multicollinearity* sorununu azaltır.
- **Yüksek Oranda Boş Değer İçerenlerin Elenmesi**: %30'dan fazla `NaN` içeren öznitelikler çıkarılır.
- **Sabit Değerli Özniteliklerin Elenmesi**: Model için hiçbir bilgi taşımayan (tek bir değere sahip) öznitelikler çıkarılır.
- **Sızıntı Kaynağı Olanların Elenmesi**: Hedef değişkeni doğrudan içeren `price actual` gibi sütunlar öznitelik setinden çıkarılır.

Bu adımlar sonunda, saatlik ve günlük modeller için kullanılacak nihai öznitelik listeleri oluşturulur.

In [28]:
corr_matrix = energy[potential_features + TARGET_COLS].corr()
cols_to_drop_redundant = set()
threshold = 0.98
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        col1, col2 = corr_matrix.columns[i], corr_matrix.columns[j]
        if col1 in potential_features and col2 in potential_features and col1 not in cols_to_drop_redundant and col2 not in cols_to_drop_redundant:
            if abs(corr_matrix.iloc[i, j]) > threshold:
                corr1_target = abs(corr_matrix.loc[col1, 'target_price_1h']) + abs(corr_matrix.loc[col1, 'target_price_next_day'])
                corr2_target = abs(corr_matrix.loc[col2, 'target_price_1h']) + abs(corr_matrix.loc[col2, 'target_price_next_day'])
                if corr1_target < corr2_target: cols_to_drop_redundant.add(col1)
                else: cols_to_drop_redundant.add(col2)

In [29]:
nan_ratios = energy[potential_features].isna().mean()
cols_to_drop_nan = set(nan_ratios[nan_ratios > 0.3].index)

cols_to_drop_constant = {col for col in potential_features if energy[col].nunique() <= 1}


In [30]:
feature_meta['drop_reason'] = None
feature_meta.loc[feature_meta['feature_name'].isin(cols_to_drop_redundant), 'drop_reason'] = 'high_corr_redundant'
feature_meta.loc[feature_meta['feature_name'].isin(cols_to_drop_nan), 'drop_reason'] = 'high_nan_ratio'
feature_meta.loc[feature_meta['feature_name'].isin(cols_to_drop_constant), 'drop_reason'] = 'constant_value'


In [31]:
feature_meta['is_clean'] = feature_meta['drop_reason'].isnull()

hourly_types = ['hourly_specific', 'common', 'fft', 'wavelet', 'weather', 'generation_entropy', 'generation_ratio', 'momentum']
feature_meta['used_in_model_1h'] = feature_meta['is_clean'] & feature_meta['feature_type'].isin(hourly_types)

daily_types = ['daily_specific', 'common', 'fft', 'wavelet', 'weather', 'generation_entropy', 'generation_ratio', 'momentum']
feature_meta['used_in_model_next_day'] = feature_meta['is_clean'] & feature_meta['feature_type'].isin(daily_types)

In [32]:
final_features_1h = feature_meta[feature_meta['used_in_model_1h']]['feature_name'].tolist()
final_features_next_day = feature_meta[feature_meta['used_in_model_next_day']]['feature_name'].tolist()

print(f"Saatlik Model İçin Kullanılacak Güvenli Özellik Sayısı: {len(final_features_1h)}")
print(f"Günlük Model İçin Kullanılacak Güvenli Özellik Sayısı: {len(final_features_next_day)}")

Saatlik Model İçin Kullanılacak Güvenli Özellik Sayısı: 122
Günlük Model İçin Kullanılacak Güvenli Özellik Sayısı: 113


In [33]:
full_drop_list = list(cols_to_drop_redundant | cols_to_drop_nan | cols_to_drop_constant | set(LEAKY_SOURCE_COLS))
energy.drop(columns=full_drop_list, inplace=True, errors='ignore')
print(f"Veriden toplam {len(full_drop_list)} sütun silindi.")

Veriden toplam 43 sütun silindi.


In [34]:
output_path = '../../data/processed/feature_metadata_2_cleaned.csv'
feature_meta.to_csv(output_path, index=False)

In [35]:
import os
import joblib
from sklearn.preprocessing import MinMaxScaler


SEQUENCE_LENGTH_1H = 24
SEQUENCE_LENGTH_NEXT_DAY = 72

TARGET_COL_1H = 'target_price_1h'
TARGET_COL_NEXT_DAY = 'target_price_next_day'

TRAIN_RATIO = 0.7
VAL_RATIO = 0.15

OUTPUT_DATA_PATH = '../../data/processed/'

os.makedirs(OUTPUT_DATA_PATH, exist_ok=True)


### 5. RNN için Veri Setinin Son Hazırlığı

Bu bölümde, temizlenmiş öznitelik seti, RNN modelinin işleyebileceği formata getirilmektedir.

#### 5.1. Son Kalan Boş Değerlerin Silinmesi
Öznitelik mühendisliği adımlarından (özellikle lag ve rolling) kaynaklanan başlangıçtaki boş değerli satırlar veri setinden tamamen temizlenir.

In [36]:
energy.dropna(inplace=True)

print(f"NaN içeren satırlar silindi. Yeni veri seti boyutu: {energy.shape}")

NaN içeren satırlar silindi. Yeni veri seti boyutu: (33280, 143)


#### 5.2. Veri Setini Kronolojik Olarak Bölme (Train-Validation-Test Split)

Zaman serisi verilerinde, modelin gelecekteki veriyi görmemesi için veri setini rastgele değil, kronolojik olarak bölmek zorunludur. Verinin ilk %70'i eğitim, sonraki %15'i validasyon ve son %15'i test için ayrılmıştır.

In [37]:
n_samples = len(energy)
train_split_idx = int(n_samples * TRAIN_RATIO)
val_split_idx = int(n_samples * (TRAIN_RATIO + VAL_RATIO))

# Veriyi DataFrame olarak böl
train_df = energy.iloc[:train_split_idx].copy()
val_df = energy.iloc[train_split_idx:val_split_idx].copy()
test_df = energy.iloc[val_split_idx:].copy()

print("Veri setleri kronolojik olarak bölündü:")
print(f"  - Eğitim Seti   : {len(train_df)} satır ({train_df.index.min()} -> {train_df.index.max()})")
print(f"  - Validasyon Seti: {len(val_df)} satır ({val_df.index.min()} -> {val_df.index.max()})")
print(f"  - Test Seti      : {len(test_df)} satır ({test_df.index.min()} -> {test_df.index.max()})")

Veri setleri kronolojik olarak bölündü:
  - Eğitim Seti   : 23296 satır (259 -> 25896)
  - Validasyon Seti: 4992 satır (25897 -> 30888)
  - Test Seti      : 4992 satır (30889 -> 36219)


#### 5.3. Veri Ölçeklendirme (Scaling)

Farklı ölçeklerdeki öznitelikler, sinir ağlarının eğitimini yavaşlatabilir veya kararsızlaştırabilir. `MinMaxScaler` kullanarak tüm öznitelikleri ve hedef değişkeni [0, 1] aralığına sıkıştırıyoruz.

**Önemli Not:** Ölçekleyici (`scaler`), sadece **eğitim verisi** üzerinde eğitilir (`.fit()`) ve ardından bu eğitilmiş ölçekleyici hem eğitim, hem validasyon hem de test verisini dönüştürmek (`.transform()`) için kullanılır. Bu, validasyon ve test setlerinden eğitim sürecine bilgi sızmasını (data leakage) engeller.

In [38]:
scaler_target_next_day = MinMaxScaler()

scaler_target_next_day.fit(train_df[[TARGET_COL_NEXT_DAY]].values)

train_df[TARGET_COL_NEXT_DAY] = scaler_target_next_day.transform(train_df[[TARGET_COL_NEXT_DAY]])
val_df[TARGET_COL_NEXT_DAY] = scaler_target_next_day.transform(val_df[[TARGET_COL_NEXT_DAY]])
test_df[TARGET_COL_NEXT_DAY] = scaler_target_next_day.transform(test_df[[TARGET_COL_NEXT_DAY]])

scaler_target_path = os.path.join(OUTPUT_DATA_PATH, 'scaler_target_next_day.joblib')
joblib.dump(scaler_target_next_day, scaler_target_path)

print(f"Hedef değişken (target) için scaler oluşturuldu ve '{scaler_target_path}' adresine kaydedildi.")
print("Train, Val ve Test setlerindeki hedef sütunlar başarıyla ölçeklendirildi.")

Hedef değişken (target) için scaler oluşturuldu ve '../../data/processed/scaler_target_next_day.joblib' adresine kaydedildi.
Train, Val ve Test setlerindeki hedef sütunlar başarıyla ölçeklendirildi.


#### 5.4. Dizilerin (Sequences) Oluşturulması

RNN modelleri, veriyi 3 boyutlu bir tensör olarak bekler: `(örnek_sayısı, zaman_adımı_sayısı, öznitelik_sayısı)`.

Bu yapı, modele "geçmiş `N` adımdaki özniteliklere bakarak bir sonraki adımı tahmin et" komutunu vermemizi sağlar. `create_sequences` fonksiyonu, 2 boyutlu veri çerçevesini bu 3 boyutlu yapıya dönüştürür. Örneğin, `SEQUENCE_LENGTH_1H = 24` ise, model bir sonraki saatin fiyatını tahmin etmek için geçmiş 24 saatin verisine bakacaktır.

In [39]:
def create_sequences(df, feature_cols, target_col, sequence_length):
    X, y = [], []
    for i in range(sequence_length, len(df)):
        if pd.isna(df.iloc[i][target_col]):
            continue
        seq_x = df[feature_cols].iloc[i-sequence_length:i].values
        seq_y = df.iloc[i][target_col]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

### 6. Veri Setlerini Son Haline Getirme ve Kaydetme

Son adım olarak, hem saatlik (`_1h`) hem de günlük (`_next_day`) modeller için ölçeklendirme ve dizilere dönüştürme işlemleri uygulanır.

Oluşturulan bu işlenmiş diziler (`.npz` formatında) ve ölçekleyiciler (`.joblib` formatında) diske kaydedilir. Bu sayede, model eğitimi yapılırken bu uzun öznitelik mühendisliği sürecini tekrar çalıştırmak gerekmeyecektir.

In [40]:
#SAATLİK MODEL


scaler_1h = MinMaxScaler()

# Scaler'ı sadece eğitim verisinin özelliklerine göre eğitiyoruz hedef sütunu ölçeklendirmeye dahil etmiyoruz.
train_features_1h = train_df[final_features_1h]
scaler_1h.fit(train_features_1h)

# Tüm setleri eğitilmiş scaler ile dönüştür
train_df.loc[:, final_features_1h] = scaler_1h.transform(train_features_1h)
val_df.loc[:, final_features_1h] = scaler_1h.transform(val_df[final_features_1h])
test_df.loc[:, final_features_1h] = scaler_1h.transform(test_df[final_features_1h])

X_train_1h, y_train_1h = create_sequences(train_df, final_features_1h, TARGET_COL_1H, SEQUENCE_LENGTH_1H)
X_val_1h, y_val_1h = create_sequences(val_df, final_features_1h, TARGET_COL_1H, SEQUENCE_LENGTH_1H)
X_test_1h, y_test_1h = create_sequences(test_df, final_features_1h, TARGET_COL_1H, SEQUENCE_LENGTH_1H)

print(f"Train sequences shape: X={X_train_1h.shape}, y={y_train_1h.shape}")
print(f"Validation sequences shape: X={X_val_1h.shape}, y={y_val_1h.shape}")
print(f"Test sequences shape: X={X_test_1h.shape}, y={y_test_1h.shape}")


hourly_data_path = os.path.join(OUTPUT_DATA_PATH, 'hourly_model_data.npz')
np.savez_compressed(
    hourly_data_path,
    X_train=X_train_1h, y_train=y_train_1h,
    X_val=X_val_1h, y_val=y_val_1h,
    X_test=X_test_1h, y_test=y_test_1h
)

scaler_1h_path = os.path.join(OUTPUT_DATA_PATH, 'scaler_1h.joblib')
joblib.dump(scaler_1h, scaler_1h_path)
print(f"Saatlik model scaler'ı '{scaler_1h_path}' adresine kaydedildi.")

Train sequences shape: X=(23272, 24, 122), y=(23272,)
Validation sequences shape: X=(4968, 24, 122), y=(4968,)
Test sequences shape: X=(4968, 24, 122), y=(4968,)
Saatlik model scaler'ı '../../data/processed/scaler_1h.joblib' adresine kaydedildi.


In [41]:
# GÜNLÜK MODEL

scaler_next_day = MinMaxScaler()

train_features_next_day = train_df[final_features_next_day]
scaler_next_day.fit(train_features_next_day)

train_df.loc[:, final_features_next_day] = scaler_next_day.transform(train_features_next_day)
val_df.loc[:, final_features_next_day] = scaler_next_day.transform(val_df[final_features_next_day])
test_df.loc[:, final_features_next_day] = scaler_next_day.transform(test_df[final_features_next_day])

# Sequences Oluşturma
X_train_next_day, y_train_next_day = create_sequences(train_df, final_features_next_day, TARGET_COL_NEXT_DAY, SEQUENCE_LENGTH_NEXT_DAY)
X_val_next_day, y_val_next_day = create_sequences(val_df, final_features_next_day, TARGET_COL_NEXT_DAY, SEQUENCE_LENGTH_NEXT_DAY)
X_test_next_day, y_test_next_day = create_sequences(test_df, final_features_next_day, TARGET_COL_NEXT_DAY, SEQUENCE_LENGTH_NEXT_DAY)

print(f"Train sequences shape: X={X_train_next_day.shape}, y={y_train_next_day.shape}")
print(f"Validation sequences shape: X={X_val_next_day.shape}, y={y_val_next_day.shape}")
print(f"Test sequences shape: X={X_test_next_day.shape}, y={y_test_next_day.shape}")

#Kaydetme
daily_data_path = os.path.join(OUTPUT_DATA_PATH, 'daily_model_data.npz')
np.savez_compressed(
    daily_data_path,
    X_train=X_train_next_day, y_train=y_train_next_day,
    X_val=X_val_next_day, y_val=y_val_next_day,
    X_test=X_test_next_day, y_test=y_test_next_day
)
print(f"Günlük model verileri '{daily_data_path}' adresine kaydedildi.")

scaler_next_day_path = os.path.join(OUTPUT_DATA_PATH, 'scaler_next_day.joblib')
joblib.dump(scaler_next_day, scaler_next_day_path)
print(f"Günlük model scaler'ı '{scaler_next_day_path}' adresine kaydedildi.")

Train sequences shape: X=(23224, 72, 113), y=(23224,)
Validation sequences shape: X=(4920, 72, 113), y=(4920,)
Test sequences shape: X=(4920, 72, 113), y=(4920,)
Günlük model verileri '../../data/processed/daily_model_data.npz' adresine kaydedildi.
Günlük model scaler'ı '../../data/processed/scaler_next_day.joblib' adresine kaydedildi.


### 7. Son Kontrol

Tüm işlemler bittikten sonra, veri setinde hala herhangi bir `NaN` değeri kalıp kalmadığı son bir kez kontrol edilir.

In [ ]:
nan_counts = energy.isna().sum()

nan_counts_with_nans = nan_counts[nan_counts > 0]

sorted_nan_counts = nan_counts_with_nans.sort_values(ascending=False)

print("Öznitelik Mühendisliği Sonrası NaN Değerleri İçeren Sütunlar:")
print("-" * 60)
display(sorted_nan_counts)

Öznitelik Mühendisliği Sonrası NaN Değerleri İçeren Sütunlar:
------------------------------------------------------------


Series([], dtype: int64)